# Preprocess Population Dataset
We preprocess data for training.The population dataset will be considered:

In [1]:
import pandas as pd
import os.path as op
from sklearn.preprocessing import LabelEncoder
import numpy as np
import seaborn as sns

In [2]:
DATA_DIR = '../../Data'

## Noisy Data :

In [3]:
population_claims = pd.read_csv(op.join(DATA_DIR, 'population/Unprocessed_data/population_claims.csv'))

In [4]:
population_claims.head()

ObjectID      PropertyID  PropertyValue                  SourceID  \
0  abu dhabi  Population2006      1000230.0         0 (68.162.248.83)   
1  abu dhabi  Population2006      1850230.0  1513217: Mohammedfairouz   
2  amsterdam  Population2006       741329.0             141597: Ilse@   
3  amsterdam  Population2006       742884.0           1300620: Krator   
4   adelaide  Population2006      1124315.0         3922171: Pirate05   

   TimeStamp  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [5]:
population_claims.shape

(49943, 5)

In [6]:
claims = population_claims.copy()

In [7]:
claims = claims[claims.duplicated(subset=["SourceID"], keep=False)]

In [8]:
claims

ObjectID      PropertyID  PropertyValue  \
1                   abu dhabi  Population2006      1850230.0   
2                   amsterdam  Population2006       741329.0   
4                    adelaide  Population2006      1124315.0   
5                      athens  Population2006      4200000.0   
8                  ames, iowa  Population2006        51557.0   
...                       ...             ...            ...   
49936  gmina kamienica polska  Population2006         5634.0   
49937          gmina kłomnice  Population2006        13821.0   
49938         gmina koniecpol  Population2006        10300.0   
49939         gmina konopiska  Population2006        10405.0   
49940          gmina kruszyna  Population2006         4899.0   

                       SourceID  TimeStamp  
1      1513217: Mohammedfairouz        NaN  
2                 141597: Ilse@        NaN  
4             3922171: Pirate05        NaN  
5             1876487: El Greco        NaN  
8               1943326: Ctjf83        NaN  
...                         ...        ...  
49936           5277842: Kotbot        NaN  
49937           5277842: Kotbot        NaN  
49938           5277842: Kotbot        NaN  
49939           5277842: Kotbot        NaN  
49940           5277842: Kotbot        NaN  

[46518 rows x 5 columns]

In [9]:
claims.SourceID.value_counts()

5512121: CapitalBot     25820
3729068: BOTijo          2805
5277842: Kotbot          1507
4373711: ArkyBot         1406
840224: Omnedon          1313
                        ...  
469625: Clpo13              2
472: Yearofthedragon        2
149061: Djsasso             2
3859960: Sfmammamia         2
157842: Angr                2
Name: SourceID, Length: 832, dtype: int64

## Convert the column of PropertyValue into integer 

In [10]:
claims.PropertyValue=claims.PropertyValue.astype("int64")

In [11]:
claims.dtypes

ObjectID          object
PropertyID        object
PropertyValue      int64
SourceID          object
TimeStamp        float64
dtype: object

## Null values 

In [12]:
claims.isnull().sum()

ObjectID             0
PropertyID           0
PropertyValue        0
SourceID             0
TimeStamp        46518
dtype: int64

## Keep sources that have more then one claim 

In [ ]:
mask = claims.groupby(['SourceID']).nunique().value < 2
source_with_single_value = list(claims.groupby(['object']).nunique().value[mask].index)

## Drop duplicates

In [13]:
claims.drop_duplicates().shape, claims.shape

((46518, 5), (46518, 5))

## True data

In [14]:
population_truths = pd.read_csv(op.join(DATA_DIR, 'population/Unprocessed_data/population_truth.csv'))

In [15]:
population_truths.head()

ObjectID      PropertyID  PropertyValue
0      cleveland, ohio  Population2000         478403
1        gary, indiana  Population2000         102746
2      flint, michigan  Population2000         124943
3  compton, california  Population2000          93493
4     washington, d.c.  Population2005         582049

In [16]:
population_truths.shape

(308, 3)

In [17]:
truths = population_truths.copy()

In [18]:
truths.dtypes

ObjectID         object
PropertyID       object
PropertyValue     int64
dtype: object

## Instead of Having "ObjectID" and "PropertyID", we join them to create one column called   "Object" and we also drop the column "TimeStamp" 

In [19]:
claims['object'] = claims.ObjectID.apply(lambda x: x.replace(',', '_').replace(' ', '')) + '_' + claims.PropertyID
claims['value'] = claims.PropertyValue
claims.drop(columns=['ObjectID', 'PropertyID', 'PropertyValue', 'TimeStamp'], inplace=True)

In [20]:
truths['object'] = truths.ObjectID.apply(lambda x: x.replace(',', '_').replace(' ', '')) + '_' + truths.PropertyID
truths['value'] = truths.PropertyValue
truths.drop(columns=['ObjectID', 'PropertyID', 'PropertyValue'], inplace=True)

## Not all the claims has their corresponding truths. We need to filter them out

In [21]:
truths[truths.object=='st.joseph_missouri_Population2000'].shape[0], claims[claims.object == 'st.joseph_missouri_Population2000'].shape[0]

(1, 0)

In [22]:
object_with_claims_and_truth = list(set(claims.object).intersection(set(truths.object)))

In [23]:
truths = truths.set_index('object').loc[object_with_claims_and_truth].reset_index()

In [24]:
claims = claims.set_index('object').loc[object_with_claims_and_truth].reset_index()

In [25]:
truths.shape, claims.shape

((292, 2), (592, 3))

## At least 2 different claims for each object

In [26]:
mask = claims.groupby(['object']).nunique().value < 2
object_with_single_value = list(claims.groupby(['object']).nunique().value[mask].index)

In [27]:
claims = claims.set_index('object').drop(object_with_single_value).reset_index()
truths = truths.set_index('object').drop(object_with_single_value).reset_index()

In [28]:
truths.shape, claims.shape

((106, 2), (283, 3))

## Data Encoding

In [29]:
object_le = LabelEncoder()
claims['object_id'] = object_le.fit_transform(claims.object)

In [30]:
claims 

object                  SourceID  \
0    berkeleyheights_newjersey_Population2000       5512121: CapitalBot   
1    berkeleyheights_newjersey_Population2000            859568: NcSchu   
2        vicksburg_mississippi_Population2000  104574: ArkansasTraveler   
3        vicksburg_mississippi_Population2000       5512121: CapitalBot   
4        vicksburg_mississippi_Population2000          0 (66.140.36.82)   
..                                        ...                       ...   
278             muncie_indiana_Population2000          620554: Harpchad   
279             muncie_indiana_Population2000       5512121: CapitalBot   
280           savage_minnesota_Population2000          1396565: RyguyMN   
281           savage_minnesota_Population2000       5512121: CapitalBot   
282           savage_minnesota_Population2000          298922: Davumaya   

       value  object_id  
0      39991          9  
1      13407          9  
2      26407        100  
3      26407        100  
4    1126407        100  
..       ...        ...  
278    67430         64  
279    65287         64  
280    21115         89  
281    21115         89  
282    25065         89  

[283 rows x 4 columns]

In [31]:
source_le = LabelEncoder()

In [32]:
claims['source_id'] = source_le.fit_transform(claims.SourceID)

In [33]:
claims

object                  SourceID  \
0    berkeleyheights_newjersey_Population2000       5512121: CapitalBot   
1    berkeleyheights_newjersey_Population2000            859568: NcSchu   
2        vicksburg_mississippi_Population2000  104574: ArkansasTraveler   
3        vicksburg_mississippi_Population2000       5512121: CapitalBot   
4        vicksburg_mississippi_Population2000          0 (66.140.36.82)   
..                                        ...                       ...   
278             muncie_indiana_Population2000          620554: Harpchad   
279             muncie_indiana_Population2000       5512121: CapitalBot   
280           savage_minnesota_Population2000          1396565: RyguyMN   
281           savage_minnesota_Population2000       5512121: CapitalBot   
282           savage_minnesota_Population2000          298922: Davumaya   

       value  object_id  source_id  
0      39991          9        103  
1      13407          9        121  
2      26407        100         28  
3      26407        100        103  
4    1126407        100         10  
..       ...        ...        ...  
278    67430         64        109  
279    65287         64        103  
280    21115         89         38  
281    21115         89        103  
282    25065         89         74  

[283 rows x 5 columns]

In [34]:
truths['object_id'] = object_le.transform(truths.object)

## Save the Data in files

In [35]:
truths.to_csv(op.join(DATA_DIR, 'population/Claims_to_work_with/truths_TF.csv'), index=False)

In [36]:
claims.to_csv(op.join(DATA_DIR, 'population/Claims_to_work_with/claims_TF.csv'), index=False)

In [37]:
truths.shape, claims.shape

((106, 3), (283, 5))